# **Tugas 1 - Crawling Data (detik.com)**

**Nama : Achmad Baharuddin Akbar**

**NIM  : 210411100001**

Crawling data adalah proses pengambilan data yang digunakan untuk mengumpulkan informasi dari situs web. proses ini dilakukan oleh program komputer yang disebut **"crawler" atau "spider,"** yang menavigasi situs web dan mengekstrak data yang diperlukan.

Pada tugas ini menggunakan **"BeautifulSoup"** sebagai library Crawler dan proses pengambilan data diambil dari website **detik.com**

langkah pertama Membuat fungsi untuk membersihkan dan memformat konten HTML yang diambil dari halaman web ***detik.com*** lebih detailnya :

- Menghapus Elemen yang Tidak Diinginkan dengan mengidentifikasi dan menghapus elemen-elemen tertentu yang dianggap tidak relevan, seperti daftar isi, tag yang tidak diinginkan, dan link sisipan.

- Menghapus Paragraf dan Span dengan kelas tertentu yang tidak diinginkan.

- Mengembalikan Teks Bersih
Setelah elemen yang tidak diinginkan dihapus, kode akan mengembalikan teks yang tersisa dengan memformatnya menjadi string yang bersih dan terpisah dengan spasi.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk membersihkan konten dari elemen-elemen yang tidak diinginkan
def clean_content(content_element):
    if content_element:
        # Hapus elemen yang berisi daftar isi
        for daftar_isi in ["collapsible"]:
            unwanted = content_element.find("div", id=daftar_isi)
            if unwanted:
                unwanted.decompose()

        # Hapus elemen yang berisi tag
        for tag_class in ["aevp", "detail__body-tag mgt-16"]:
            unwanted = content_element.find_all("div", class_=tag_class)
            for el in unwanted:
                el.decompose()

        # Hapus elemen yang berisi link sisipan
        link_sisip = content_element.find_all("table", class_="linksisip")
        for table in link_sisip:
            table.decompose()

        # Hapus elemen paragraf dan span dengan class 'para_caption'
        unwanted_paragraphs = content_element.find_all(["p", "span"], class_="para_caption")
        for para in unwanted_paragraphs:
            para.decompose()

        # Kembalikan teks yang tersisa
        return content_element.get_text(separator=' ', strip=True).strip()

    return "Content Not Found"


Selanjutnya untuk mengumpulkan data berita dari situs web Detik.com dan menyimpannya dalam format CSV.

- Proses pengambilan data dengan Fungsi get_data yang digunakan untuk mengambil artikel berita dari halaman yang ditentukan. Kode ini bertujuan crawl URL, mem-parsing HTML untuk menemukan artikel, dan kemudian mengambil detail artikel (judul, tanggal, isi berita, kategori) dari setiap link artikel yang ditemukan.

- Pembersihan Konten/data menggunakan fungsi clean_content untuk menghapus elemen yang tidak diinginkan.

- Save Data artikel (judul, tanggal, isi berita, kategori) disimpan dalam list dan kemudian dikumpulkan dalam DataFrame Pandas.

In [2]:
# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori, min_articles_per_category):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len([k for k in kategori_list if k == kategori]) >= min_articles_per_category:
            return  # Menghentikan proses jika jumlah artikel sudah mencapai minimum yang diinginkan

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = content_element.text.strip() if content_element else "Content Not Found"

        # Bersihkan konten menggunakan fungsi clean_content
        content = clean_content(content_element)


        judul.append(title)
        tanggal.append(date)
        isi.append(content)
        # url_list.append(link)
        kategori_list.append(kategori)

        if len(judul) <= 10:
          print(title)
        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

# Membuat list url dan kategori yang akan di-crawl
base_urls = [
    "https://health.detik.com/berita-detikhealth/indeks",
    "https://food.detik.com/indeks",
]
categories = [
    "Kesehatan",
    "Kuliner",
]

# Inisialisasi list untuk menyimpan data
judul = []
tanggal = []
isi = []
kategori_list = []

# Batas minimal artikel per kategori
min_articles_per_category = 50

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    page = 1
    while len([k for k in kategori_list if k == category]) < min_articles_per_category:
        url = f"{base_url}/{page}"
        get_data(url, category, min_articles_per_category)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman
        page += 1

# Membuat dataframe dari list data
df = pd.DataFrame({"judul": judul, "tanggal": tanggal, "isi": isi, "kategori": kategori_list})

# Menyimpan dataframe ke file csv
df.to_csv("data_berita_detik.csv", index=False)

# Menampilkan dataframe
# print(df)

Jangan Sembarangan! Waspadai Pemicu Cedera saat Lari yang Sering Dialami Pemula
Keseringan Pakai TWS? Waspada Gangguan Pendengaran NIHL, Ini Tandanya
Jogging Pakai TWS Aman Nggak Sih? Ini Plus-Minusnya Menurut Dokter
5 Kebiasaan Simpel yang Bisa Bikin Panjang Umur, Salah Satunya Menikmati Makanan
Rajin Sit-up Saja Nggak Bikin Lemak di Perut Hilang, Ini yang Kerap Terlupakan
Waduh! Peneliti di China Temukan Virus Baru Akibat Kutu, Efeknya Bisa ke Otak
Banjir Keringat saat Olahraga Tanda Tubuh Bakar Lemak? Begini Faktanya
Kemendikbudristek Turun Tangan Urus Bullying PPDS Undip, Bentuk Tim Pencari Fakta
Terbukti Lewat Studi, Doyan Traveling Ternyata Bisa Bikin Panjang Umur!
9 Kelompok Orang yang Tidak Disarankan Minum Kopi


Data yang diambil terdiri dari :
- Judul
- Isi berita
- Tanggal
- Kategori

Pada tahap ini menampilkan data tersebut dan diambil berdasarkan kategori
***Kesehatan*** dan ***Kuliner*** dan hanya mengambil 100 sampel data crawl

Berikut untuk 5 sampel data berdasarkan kategori ***Kesehatan***

In [7]:
df=pd.read_csv("data_berita_detik.csv")
df.head(5)

,judul,tanggal,isi,kategori
0,Jangan Sembarangan! Waspadai Pemicu Cedera saa...,"Sabtu, 07 Sep 2024 19:00 WIB",Jakarta - Olahraga lari dalam beberapa waktu t...,Kesehatan
1,Keseringan Pakai TWS? Waspada Gangguan Pendeng...,"Sabtu, 07 Sep 2024 18:00 WIB",Jakarta - Adanya teknologi earphone nirkabel a...,Kesehatan
2,Jogging Pakai TWS Aman Nggak Sih? Ini Plus-Min...,"Sabtu, 07 Sep 2024 17:00 WIB",Jakarta - Tren penggunaan earphone nirkabel at...,Kesehatan
3,5 Kebiasaan Simpel yang Bisa Bikin Panjang Umu...,"Sabtu, 07 Sep 2024 14:00 WIB",Jakarta - Hidup sehat dengan umur yang panjang...,Kesehatan
4,Rajin Sit-up Saja Nggak Bikin Lemak di Perut H...,"Sabtu, 07 Sep 2024 13:00 WIB",Jakarta - Spesialis kedokteran olahraga dr And...,Kesehatan


Berikut untuk 5 sampel data berdasarkan kategori ***Kuliner***

In [8]:
df.tail(5)

,judul,tanggal,isi,kategori
95,Resep Ayam Goreng Saus Mangga ala Restoran yan...,"Jumat, 06 Sep 2024 10:00 WIB",Tentang Bahan Langkah Jakarta - Ayam goreng ya...,Kuliner
96,Ramen Kaki Lima di Yogyakarta hingga Menu Maka...,"Jumat, 06 Sep 2024 09:30 WIB",Jakarta - Rekomendasi ramen kaki lima di Yogya...,Kuliner
97,"3 Resep Soto Ayam Jawa Paling Populer, Bening ...","Jumat, 06 Sep 2024 09:00 WIB",Tentang Bahan Langkah Jakarta - Soto ayam berk...,Kuliner
98,'Food Ingredients Asia' Hadirkan Teknologi Pan...,"Jumat, 06 Sep 2024 08:30 WIB",Jakarta - Pameran Food Ingredients Asia kembal...,Kuliner
99,"Reza Rahadian Ternyata Pencinta Kopi, Ini Favo...","2,810 Views | Jumat, 06 Sep 2024 07:50 WIB",Reza Rahadian ternyata seorang penggila kopi. ...,Kuliner
